0. Initialize

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import sys
import glob
import datetime
import os

In [0]:
sys.path.append('drive/My Drive/Colab Notebooks')

In [0]:
from astropy.io import fits
from astropy.wcs import WCS
import cupy  as cp
import numpy as np

In [0]:
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval, ImageNormalize

In [0]:
from eclair import FitsContainer, reduction, fixpix, imalign, imcombine

In [0]:
dirname = '/content/drive/My Drive/Colab Notebooks/test'

fitslist = ['raw%02d.fits'%i for i in range(10)]

dark = 'dark.fits'
flat = 'flat.fits'
bpmask = 'bpmask.fits'

output = 'combine.fits'

In [0]:
xy0 = np.array([[450,450],[450,650],[650,450],[650,650]])
slices = dict(xmin=52, xmax=1072, ymin=2, ymax =1022)

1. Data Loading

In [0]:
os.chdir(dirname)

In [0]:
fc = FitsContainer(fitslist)
fc.setslice(**slices)
fc.load()

In [0]:
npdark = fits.getdata(dark).astype('f4')
npflat = fits.getdata(flat).astype('f4')
npbpm = fits.getdata(bpmask).astype('f4')

cpdark = cp.array(npdark)
cpflat = cp.array(npflat+(npflat==0.0).astype('f4'))
cpbpm = cp.sign(cp.array(npbpm))

2. Get shift

In [0]:
wcs = {f:WCS(fc.header[f]) for f in fitslist}

In [0]:
def getshift(idx):
  ad0 = wcs[fitslist[idx]].wcs_pix2world(xy0,1)
  shift = np.empty([len(fitslist),2],dtype='f4')
  for i, f in enumerate(fitslist):
    xy = wcs[f].wcs_world2pix(ad0,1)
    shift[i,...] = (xy0-xy).mean(axis=0)
    
  return shift

In [0]:
shift = getshift(0)
idx = np.linalg.norm(shift-np.median(shift,axis=0),axis=1).argmin()
shift = getshift(idx)

3. Reduction

In [0]:
bias = [fc.header[f]['PEDLEVEL'] for f in fitslist]
cpbias = cp.array(bias,dtype='f4')

In [0]:
fc.data = reduction(fc.data, cpbias[:,cp.newaxis,cp.newaxis], cpdark, cpflat)

In [0]:
fc.data = fixpix(fc.data, cpbpm)

4. Align

In [0]:
fc.data = imalign(fc.data, shift)

5. Combine

In [0]:
imcombine(fitslist, fc.data, output, overwrite=True)

In [0]:
ls combine.fits

6. Result

In [0]:
data = fits.getdata(output)
vrange = ZScaleInterval().get_limits(data)
data = data.clip(*vrange)
y_len, x_len = data.shape

plt.figure()
plt.imshow(data)
plt.xlim(0,x_len)
plt.ylim(0,y_len)
plt.colorbar()
plt.show()